<a href="https://colab.research.google.com/github/Rekt77/kisa_insuretech/blob/master/torch_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.init

# gpu사용
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(1000)
if device == "cuda":
  torch.cuda.manual_seed(1000)

batch_size=100
learning_rate=0.001
epochs=15

In [0]:
mnist_train = datasets.MNIST(root="/content/drive/My Drive/insuretech/MNIST_CNN/",
                             train=True,
                             download=True,
                             transform=transforms.ToTensor())

mnist_test = datasets.MNIST(root="/content/drive/My Drive/insuretech/MNIST_CNN/",
                             train=False,
                             download=True,
                             transform=transforms.ToTensor())


In [0]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [0]:
# CNN Model (2-layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [0]:
# CNN모델이 gpu를 사용하도록 만들기
model = CNN().to(device)

In [0]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

NameError: ignored

In [0]:
total_batch = len(data_loader)

for epoch in range(15):

  # loss 값을 담을 변수 0으로 초기화
  avg_cost = 0

  # img는 input, label은 정답지
  for img,label in data_loader:
    img = img.to(device)
    label = label.to(device)

    # 선형회귀에서 했던 코드
    optimizer.zero_grad()
    hypothesis = model(img)
    cost = criterion(hypothesis, label)

    cost.backward()
    optimizer.step()

    avg_cost += cost/total_batch

  print("Epoch:{} cost={}".format(epoch+1, avg_cost))

In [0]:
# 정확도 테스트
with torch.no_grad():

    # 테스트 데이터 불러와서 1렬 뉴런 형태로 불러오기
    img_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    label_test = mnist_test.test_labels.to(device)
    
    # 학습된 CNN모델에 삽입
    prediction = model(img_test)
    # 테스트 라벨과 prediction의 라벨을 비교해서 값이 True인것만 추출
    correct_prediction = torch.argmax(prediction, 1) == label_test

    # 모은 데이터의 평균을 내서 정확도를 추출
    accuracy = correct_prediction.float().mean()

    # 정확도 프린트
    print('Accuracy:', accuracy.item())